# 통계 기반 기법

1) 간단한 전처리  
2) word_to_id, id_to_word로 분리 
3) 분포 가설에 따라 co-occurrence matrix(동시 발생 행렬)로 변환
   1) 동시 발생 행렬: 특정 단어의 window 내에 존재하는 단어들의 빈도
4) 코싸인 유사도 기반 단어와 유사한 단어 추출 가능

In [8]:
import numpy as np
text = "You say goodbye and I say hello."

In [11]:
def preprocess(text):
    text = text.lower().replace(".", " .")
    words = text.split()

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    return corpus, word_to_id, id_to_word

In [12]:
corpus, word_to_id, id_to_word = preprocess(text)

In [13]:
print(corpus, word_to_id, id_to_word)

[0 1 2 3 4 1 5 6] {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6} {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


### 분포 가설  
단어의 의미는 주변 단어에 의해 형성된다. -> 단어 주변 window 살펴서 단어를 벡터화 가능

In [14]:
import sys
sys.path.append("..")
import numpy as np
corpus, word_to_id, id_to_word = preprocess(text)

In [15]:
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


window 1이라 가정하고 맨 앞 단어 you의 맥락에 포함되는 건 'say' 하나 뿐  
You say goodbye and I say hello . 기준에서 window에 해당하는 걸로 표현하자면    
you -> [0 1 0 0 0 0 0]

say의 경우 한 문장 내 2번 등장하는데 각 윈도우로 생각해보면  
say -> [1 0 1 0 1 1 0]

이러한 작업을 모든 단어에 대해서 진행하고 나타낸 행렬을 동시 발생 행렬(co-occurrence matrix)라고 한다.

In [16]:
def create_co_matrix(corpus, vocab_size, window_size = 1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype = np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    
    return co_matrix
                

In [17]:
create_co_matrix(corpus, len(word_to_id), 1)

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

In [18]:
def cos_similarity(x, y, eps = 1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [28]:
# word_matrix는 co-occurence matrix를 말함
def most_similar(query, word_to_id, id_to_words, word_matrix, top=5):
    if query not in word_to_id:
        print("%s is not found" % query)
        return
    
    print("\n[query] " + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # cosine similarity
    vocab_size = len(id_to_words)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # cosine similarity 기준 내림순 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_words[i] == query:
            continue
        print(" %s: %s" % (id_to_words[i], similarity[i]))

        count += 1
        if count >= top:
            return


In [31]:

C = create_co_matrix(corpus, len(word_to_id), 1)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))

most_similar('you', word_to_id, id_to_word, C)

0.7071067691154799

[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
